In [1]:
import tensorflow as tf
import numpy as np
import PIL.Image as PILImage
import scipy
import matplotlib.pyplot as plt
from src.hyperparameters import *
from src.models import *
from src.utils import *
import src.mnist_preprocess
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
data = src.mnist_preprocess.load_mnist_color()
print(data.shape)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting mnist\t10k-images-idx3-ubyte.gz
Extracting mnist\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
(55000, 64, 64, 3)


In [3]:
tf.reset_default_graph()
sess = tf.Session()

real_data_placeholder = tf.placeholder('float', shape=[batch_size, image_size, image_size, 3])
z_placeholder = tf.placeholder(tf.float32, [batch_size, noise_dimension])

disc_output_real = forward_pass_discriminator(real_data_placeholder)
gen_output = forward_pass_generator(z_placeholder)
disc_output_fake = forward_pass_discriminator(gen_output, reuse='False')

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
real_image_loss = l1_loss(real_data_placeholder, disc_output_real)
fake_image_loss = l1_loss(gen_output, disc_output_fake)
discriminator_loss = discriminator_loss(real_image_loss, fake_image_loss, kt_equilbrium_term)
generator_loss = fake_image_loss
kt_p = kt_step(kt_equilbrium_term, lambda_kt_learning_rate, gamma_diversity_ratio, real_images_loss, fake_images_loss)
convergence = convergence(real_image_loss, fake_image_loss, gamma_diversity_ratio)

NameError: name 'discriminator_loss' is not defined

In [ ]:
tvars = tf.trainable_variables()
disc_vars = [var for var in tvars if 'dec' in var.name]
gen_vars = [var for var in tvars if 'gen' in var.name]

In [ ]:
adam = tf.train.AdamOptimizer(learning_rate)
dis_opt = adam.minimize(discriminator_loss, var_list=disc_vars)
gen_opt = adam.minimize(generator_loss, var_list=gen_vars)

In [ ]:
sess.run(tf.global_variables_initializer())
iterations=1
for i in range(iterations):
    z_batch = np.random.uniform(-1,1,size=[batch_size, noise_dimension])
    
    start_batch = i*batch_size
    end_batch = start_batch+batch_size
    real_image_batch = data[start_batch:end_batch, :,:,:]
    
    _, total_disc_loss=sess.run([dis_opt, discriminator_loss], feed_dict={z_placeholder: z_batch, real_data_placeholder: real_image_batch})
    _, total_gen_loss=sess.run([gen_opt, generator_loss], feed_dict={z_placeholder: z_batch})
    
    m_global = sess.run([convergence], feed_dict={z_placeholder: z_batch, real_data_placeholder: real_image_batch})
    
    print("Step: {}, Convergence: {} ,Discriminator Loss: {}, Generator Loss {}".format(i, m_global, total_disc_loss, total_gen_loss))

In [ ]:
sample_image=forward_pass_generator(z_placeholder, reuse=True)
z_batch = np.random.uniform(-1,1,size=[16, noise_dimension])
temp = (sess.run(sample_image, feed_dict={z_placeholder: z_batch}))
# my_i = temp.squeeze()
plt.imshow(temp[2])
plt.show()